In [208]:
import git
from git import Repo

from unidiff import PatchSet

import subprocess
import json
import pandas as pd
import numpy as np
import matplotlib
import sys
import matplotlib.pyplot as plt
import sklearn

from mutester.reordering_evaluation import ReorderingEvaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier

In [346]:
# Covariance Matrix 
mutants_and_tests = pd.read_pickle('20200710-204704_httpie_full_with_context.pkl')
mutants_and_tests = mutants_and_tests.loc[mutants_and_tests['test_id'] < 100].loc[mutants_and_tests['mutant_id'] < 1000]
mutants_and_tests.reset_index()
mutants_and_tests["outcome"] = mutants_and_tests["outcome"].astype('bool')
mutants_and_tests["outcome"]
display(mutants_and_tests)

,mutant_id,contains_branch_mutant,contains_equality_comparison_mutant,contains_loop_mutant,contains_math_operands_mutant,current_line,line_number_changed,modified_file_path,modified_method,previous_line,...,setup_outcome,setup_duration,call_outcome,call_duration,teardown_outcome,teardown_duration,contains_branch_execution,contains_loop_execution,contains_math_operands_execution,contains_equality_comparison_execution
37620,1.0,1.0,1.0,1.0,1.0,"ef humanize_bytes(n, precision=3):",22.0,httpie/utils.py,humanize_bytes,,...,True,0.042428,True,0.009426,True,0.000530,None,None,None,None
37621,1.0,1.0,1.0,1.0,1.0,"ef humanize_bytes(n, precision=3):",22.0,httpie/utils.py,humanize_bytes,,...,True,0.000479,True,0.009316,True,0.000555,None,None,None,None
37622,1.0,1.0,1.0,1.0,1.0,"ef humanize_bytes(n, precision=3):",22.0,httpie/utils.py,humanize_bytes,,...,True,0.000870,True,0.011497,True,0.000390,None,None,None,None
37623,1.0,1.0,1.0,1.0,1.0,"ef humanize_bytes(n, precision=3):",22.0,httpie/utils.py,humanize_bytes,,...,True,0.000708,True,0.006923,True,0.000267,None,None,None,None
37624,1.0,1.0,1.0,1.0,1.0,"ef humanize_bytes(n, precision=3):",22.0,httpie/utils.py,humanize_bytes,,...,True,0.000407,True,0.004746,True,0.000227,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293145,971.0,1.0,1.0,0.0,1.0,self.args.auth_type = None,187.0,httpie/cli/argparser.py,_process_auth,,...,True,0.000313,True,0.000495,True,0.000208,None,None,None,None
293146,971.0,1.0,1.0,0.0,1.0,self.args.auth_type = None,187.0,httpie/cli/argparser.py,_process_auth,,...,True,0.000300,True,0.000465,True,0.000193,None,None,None,None
293147,971.0,1.0,1.0,0.0,1.0,self.args.auth_type = None,187.0,httpie/cli/argparser.py,_process_auth,,...,True,0.000300,True,0.000466,True,0.000193,None,None,None,None
293148,971.0,1.0,1.0,0.0,1.0,self.args.auth_type = None,187.0,httpie/cli/argparser.py,_process_auth,,...,True,0.000338,True,0.000528,True,0.000237,None,None,None,None


# Data Preparation

In [347]:
feature_columns = ["current_line", "line_number_changed", "modified_file_path", "previous_line", "test_id", "name", "filepath", "modified_method", "mutant_id"]
#feature_columns = ["current_line", "line_number_changed", "modified_file_path", "previous_line", "test_id", "name", "filepath"]
X = mutants_and_tests[feature_columns]
y = mutants_and_tests["outcome"]

# Throw out nan values:
X = X.loc[mutants_and_tests[feature_columns].isnull().any(axis=1) == False]
y = mutants_and_tests["outcome"].loc[mutants_and_tests[feature_columns].isnull().any(axis=1) == False]
y = y.astype('bool')
len(X)

# Encode stuff we want to encode:
encoded_columns = ["modified_method", "modified_file_path", "name", "filepath", "current_line", "previous_line"]
# encoded_columns = ["name", "filepath", "modified_file_path", "current_line", "previous_line"]
# Encoding
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X_enc = enc.fit_transform(X[encoded_columns])
i = 0
for column_name in encoded_columns:
    X[column_name] = X_enc[:,i]
    i += 1


split_mutant = X['mutant_id'].max() * 0.7
print("split_mutant is " + str(split_mutant))
# TODO: this has to split of whole
train = X.loc[X['mutant_id'] < split_mutant]
test = X.loc[X['mutant_id'] >= split_mutant]

X_train = X.loc[X['mutant_id'] < split_mutant]
y_train = y.loc[X['mutant_id'] < split_mutant]

X_test = X.loc[X['mutant_id'] >= split_mutant]
y_test = y.loc[X['mutant_id'] >= split_mutant]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=420, shuffle=False)
X_train


split_mutant is 699.3


,current_line,line_number_changed,modified_file_path,previous_line,test_id,name,filepath,modified_method,mutant_id
37620,503.0,22.0,9.0,0.0,0.0,53.0,0.0,33.0,1.0
37621,503.0,22.0,9.0,0.0,1.0,71.0,0.0,33.0,1.0
37622,503.0,22.0,9.0,0.0,2.0,72.0,0.0,33.0,1.0
37623,503.0,22.0,9.0,0.0,3.0,67.0,0.0,33.0,1.0
37624,503.0,22.0,9.0,0.0,4.0,69.0,0.0,33.0,1.0
...,...,...,...,...,...,...,...,...,...
193796,95.0,67.0,5.0,0.0,95.0,23.0,3.0,45.0,526.0
193797,95.0,67.0,5.0,0.0,96.0,27.0,3.0,45.0,526.0
193798,95.0,67.0,5.0,0.0,97.0,25.0,3.0,45.0,526.0
193799,95.0,67.0,5.0,0.0,98.0,24.0,3.0,45.0,526.0


In [348]:
class Reorderer:
    
    
    def fit(self, X_train, y_train):
        raise NotImplementedError
    
    # Task: make an ordering for every mutant in X_test
    def predict(self, X_test):
        raise NotImplementedError

In [349]:
# Just uses a linear ordering
class NaiveReorderer:
    def fit(self, X_train, y_train):
        pass
    
    def predict(self, X_test):
        test_ids = list(X_test.groupby('test_id').count().index)
        orderings = X_test.groupby('mutant_id').count()
        orderings['order'] = None
        orderings['order'] = orderings['order'].astype('object')
        for row in orderings.itertuples():
            orderings.at[row.Index, 'order'] = test_ids
        return orderings

In [350]:
# Executes the most flaky tests first
class AverageReorderer:
    def fit(self, X_train, y_train):
        # Count how often the test failed in the 'mutant_id' column
        X_train = X_train.copy()
        X_train['outcome'] = y_train
        sorted_test_ids = X_train.groupby(['test_id', 'outcome']).count().loc[(slice(None), False), :].sort_values('mutant_id', ascending=False).reset_index()
        # This is directly how we want our permutation to be:
        self.ordering = list(sorted_test_ids['test_id'])
        
    def predict(self, X_test):
        orderings = X_test.groupby('mutant_id').count()
        orderings['order'] = None
        orderings['order'] = orderings['order'].astype('object')
        for row in orderings.itertuples():
            orderings.at[row.Index, 'order'] = self.ordering
        return orderings
        

In [351]:
X_train

,current_line,line_number_changed,modified_file_path,previous_line,test_id,name,filepath,modified_method,mutant_id
37620,503.0,22.0,9.0,0.0,0.0,53.0,0.0,33.0,1.0
37621,503.0,22.0,9.0,0.0,1.0,71.0,0.0,33.0,1.0
37622,503.0,22.0,9.0,0.0,2.0,72.0,0.0,33.0,1.0
37623,503.0,22.0,9.0,0.0,3.0,67.0,0.0,33.0,1.0
37624,503.0,22.0,9.0,0.0,4.0,69.0,0.0,33.0,1.0
...,...,...,...,...,...,...,...,...,...
193796,95.0,67.0,5.0,0.0,95.0,23.0,3.0,45.0,526.0
193797,95.0,67.0,5.0,0.0,96.0,27.0,3.0,45.0,526.0
193798,95.0,67.0,5.0,0.0,97.0,25.0,3.0,45.0,526.0
193799,95.0,67.0,5.0,0.0,98.0,24.0,3.0,45.0,526.0


In [352]:
# This is magic: It will first predict using a binary predictor, and then put the tests in front that you said are gonna fail.
# TODO: It will even help you more: it will sort your predicted failures after duration, so that you get short durations for the first test failing
# TODO: It should probably also weight the predicted not failing tests after the average
class BinaryPredictionReorderer:
    def __init__(self, predictor):
        # This predictor should be in the 
        self.predictor = predictor
    
    
    def fit(self, X_train, y_train):
        self.predictor.fit(X_train, y_train)
        
    def predict(self, X_test):
        X_test = X_test.copy()
        orderings = X_test.groupby('mutant_id').count()
        orderings['order'] = None
        orderings['order'] = orderings['order'].astype('object')
        self.prediction = self.predictor.predict(X_test)
        X_test['outcome_prediction'] = self.prediction
        for row in orderings.itertuples():
            predictions_for_mutant = X_test.loc[X_test['mutant_id'] == row.Index]
            # print(predictions_for_mutant.loc[predictions_for_mutant['outcome_prediction'] == False]['test_id'])
            fail_predictions = predictions_for_mutant.loc[predictions_for_mutant['outcome_prediction'] == False]['test_id']
            success_predictions = predictions_for_mutant.loc[predictions_for_mutant['outcome_prediction'] == True]['test_id']
            orderring = list(fail_predictions.append(success_predictions))
            #print(len(orderring))
            orderings.at[row.Index, 'order'] = orderring
            #print(list(fail_predictions.append(success_predictions)))
        return orderings
        

In [353]:
nr = NaiveReorderer()
naive = nr.predict(mutants_and_tests)
ar = AverageReorderer()
ar.fit(X_train, y_train)
aver = ar.predict(X_test)

In [354]:
# Decision Tree
from sklearn import tree
clf = tree.DecisionTreeClassifier(ccp_alpha=0.001)

bpr_tree = BinaryPredictionReorderer(tree.DecisionTreeClassifier(ccp_alpha=0.001))
bpr_tree.fit(X_train, y_train)
bpr_tree_order = bpr_tree.predict(X_test)

In [355]:
# Forest

bpr_forest = BinaryPredictionReorderer(ExtraTreesClassifier(n_estimators=250,
                              random_state=420))
bpr_forest.fit(X_train, y_train)
bpr_forest_order = bpr_forest.predict(X_test)

In [358]:
(bpr_forest_order == bpr_tree_order)['order'].values.all()

False

## Evaluation

In [359]:
def series_analysis(name, series):
    print(name + ': ')
    print('Avg: {:.8f} Min: {:01.8f}, Max: {:01.8f}'.format(np.mean(series), np.min(series), np.max(series)))

def ordering_evaluation(ordering, mutants_and_tests):
    # TODO: Make this nicely from the class (dataclass?)
    test_count = mutants_and_tests.groupby('test_id').count().shape[0]
    print(test_count)
    apfd = []
    first_failing_duration = []
    last_test_failing_duration = []
    for row in ordering.itertuples():
        if row.Index % 50 == 0:
            print('.', end='')
        mutant_executions = mutants_and_tests.loc[mutants_and_tests['mutant_id'] == row.Index]
        # Only execute the metrics if we have at least one failure
        if mutant_executions['outcome'].values.all() == False:
            order = ordering.loc[row.Index].order
            #print(order)
            #if len(order) != test_count:
            #    print('Not a full ordering was specified, skipping...')
            #    continue
            r = ReorderingEvaluation(order, mutant_executions)
            apfd.append(r.APFD())
            first_failing_duration.append(r.first_failing_duration())
            last_test_failing_duration.append(r.last_test_failing_duration())
            #print(r.first_failing_duration())
    print('----- Analysis -------')
    # print(apfd)
    series_analysis('apfd', apfd)
    series_analysis('first_failing_duration', first_failing_duration)
    series_analysis('last_test_failing_duration', last_test_failing_duration)
    
        
ordering_evaluation(aver, mutants_and_tests)
ordering_evaluation(naive, mutants_and_tests)
ordering_evaluation(bpr_tree_order, mutants_and_tests)
ordering_evaluation(bpr_forest_order, mutants_and_tests)

100
....----- Analysis -------
apfd: 
Avg: 0.87381579 Min: 0.42607527, Max: 0.98924731
first_failing_duration: 
Avg: 0.00066967 Min: 0.00040364, Max: 0.00234795
last_test_failing_duration: 
Avg: 0.35731511 Min: 0.00089550, Max: 2.04205632
100
............----- Analysis -------
apfd: 
Avg: 0.53010876 Min: 0.23833333, Max: 0.70900000
first_failing_duration: 
Avg: 3.22115185 Min: 0.06738138, Max: 4.75023365
last_test_failing_duration: 
Avg: 8.57146602 Min: 7.14886117, Max: 9.20491195
100
....----- Analysis -------
apfd: 
Avg: 0.75857700 Min: 0.32500000, Max: 0.99000000
first_failing_duration: 
Avg: 0.08257277 Min: 0.00038624, Max: 0.45222020
last_test_failing_duration: 
Avg: 3.10204149 Min: 0.00089550, Max: 8.97350240
100
....----- Analysis -------
apfd: 
Avg: 0.83085147 Min: 0.38666667, Max: 0.99000000
first_failing_duration: 
Avg: 0.00284970 Min: 0.00038624, Max: 0.31809735
last_test_failing_duration: 
Avg: 3.27684193 Min: 0.00089550, Max: 8.97350240


In [360]:
mutant_executions = mutants_and_tests.loc[mutants_and_tests['mutant_id'] == 2]
mutant_executions
r = ReorderingEvaluation(o.loc[2].order, mutant_executions)

In [361]:
print(r.APFD())
print(r.first_failing_duration())
print(r.last_test_failing_duration())

0.9027777777777778
4.251628637313843


AssertionError: The provided dataframe contains more than one entry for the given test_id

In [138]:
ordering

0      383.0
1      225.0
2      388.0
3      387.0
4      386.0
       ...  
463    273.0
464    358.0
465    357.0
466    274.0
467    275.0
Name: test_id, Length: 468, dtype: float64